In [ ]:
import json
import safe_eval as se
from attrdict import AttrDict

res_dir = "../result/baseline/"

model_list = [
    # 'gpt-3.5-turbo-0613',
    'gpt-4-0613',
    # 'claude-v2',
    # 'gemini',
    # 'llama-api-7b',
]


ps_list = [
    # 'vitc', 
    # 'vitc_alphabet',
    # 'vitc_letters',
    # 'vitc_cards',
    # 'vitc_puzzle',
    # 'vitc_keyboard'

    # 'vitc-h-alphabet',
    # 'vitc-h-cards',
    # 'vitc-h-letters',
    # 'vitc-h-keyboard',
    # 'vitc-h-puzzle',
    # 'vitc-h-gen'
    
    # 'vitc-v-alphabet',
    # 'vitc-v-cards',
    # 'vitc-v-letters',
    # 'vitc-v-keyboard',
    # 'vitc-v-puzzle',
    # # 'vitc_vertical', # v-gen
    # 'vitc-v-gen'

    "vitc-h-block",
    "vitc-h-roman",
    "vitc-h-xchartri",
    "vitc-h-ghoulish",
    "vitc-h-hollywood",

]
defense = None
num_q = 50

for model in model_list:
    print(f"========{model}=======")
    datas = [None for _ in range(len(ps_list))]
    for idx, ps in enumerate(ps_list):
        file_path = res_dir + f"ps-{ps}_att-gpt-4-0613_tar-{model}"
        if defense is not None:
            file_path += f"_def-{defense}"
        file_path += ".json"
        with open(file_path, "r") as f:
            datas[idx] = json.load(f)

    merged = []
    # print(datas)
    for i in range(num_q):
        merged_resp_list = []
        merged_score_list = []
        for j in range(len(ps_list)):
            merged_resp_list.extend(datas[j][i]['resp_list'])
            merged_score_list.append(datas[j][i]['score'])
        
        
        merged_entry = {
            'resp_list':merged_resp_list,
            'score': max(merged_score_list)
        }
        merged.append(merged_entry)
    
    # ss eval
    se.ss_eval_handler(merged, 'adaptive')

    # gpt eval
    args = AttrDict({
        'mode': 4,
        'tmodel': model,
        'mp':1
    })
    se.gpt_eval_handler(merged, args)

    